In [30]:
import networkx as nx
import random

In [31]:
#simulation parameters 
i = 0
k = 3 #defined as 3 for its default value and is dependent of the amount of budget the city is willing to invest in new roads
f = 0.8 # shrinkage factor default value
p = .1 # average connectivity of nodes across the network with .05 as its fixed value
N =60 #fixed number of location inside the network
L_min = 5 #minimum road length
L_max = 25 #maximum road length
T = 100 # number of trips that would be generated at each clock tick

In [32]:
#function to generate random trips thoughout N number of nodes
#lets list them as touples for an (X,Y) X-start Y-end
def generate_random():
    trips = []
    for _ in range(T):#range in 100 trips
        start = random.randint(0, N-1)#satart and end at random int from 0-99
        end = random.randint(0, N-1)
        while start == end:# only if by chance while start equals end
            end = random.randint(0, N-1) # end gets anoter chance of initialization
        trips.append((start, end))# append to the empty list trips[] with (X,Y) i.e list of tuples
    return trips # return the list of trips

In [33]:

def compute_trip_count(graph, X, Y):
    # Placeholder function
    return random.randint(50, 150)  # Random count for demonstration


In [34]:
 #computing benifit matrix
def compute_benefit_matrix(graph):
    benefit_matrix = []
    for i in range(N):# for i/j in range of N (60 nodes) iterating through the graph
        for j in range(i+1, N):
            if not graph.has_edge(i, j):# if graph has the following edge when iterating
                benefit = compute_benefit(graph, i, j)# then benifit equals the function call with the specified parameters
                benefit_matrix.append((i, j, benefit))# append to the empthy list
    benefit_matrix.sort(key=lambda x: x[2], reverse=True)# sort the benifit matrix list
    return benefit_matrix# return the list


In [35]:
#2. Compute benifits matrix Benifit(X,Y) {benifit would be the how much a road benifits the network to decrease path}
# graph: NetworkX graph
# X,Y the two selected nodes representing different locations 
def compute_benefit(graph, X, Y):
    shortest_Dist = nx.shortest_path_length(graph, source=X, target=Y)
    direct_Dist = shortest_Dist * f
    benefit = (shortest_Dist - direct_Dist) * (compute_trip_count(graph, X, Y) + compute_trip_count(graph, Y, X))
    neighbor_X = set(graph.neighbors(X))
    neighbor_Y = set(graph.neighbors(Y))
    for n1 in neighbor_X:
        if n1 != Y:
            benefit += max(nx.shortest_path_length(graph, source=X, target=n1) - shortest_Dist - nx.shortest_path_length(graph, source=Y, target=n1), 0) * compute_trip_count(graph, X, n1)
    for n2 in neighbor_Y:
        if n2 != X:
            benefit += max(nx.shortest_path_length(graph, source=Y, target=n2) - shortest_Dist - nx.shortest_path_length(graph, source=X, target=n2), 0) * compute_trip_count(graph, Y, n2)
    return benefit


In [36]:
#7.function to update benifit matrix after building a road with the highest benifit
#L,M is the nodes where the road is being built to connect them
#benifit matrix: list of touples (X,Y,benifit) i.e.potential road connections and the benifit factor
def update_matrix(graph, L, M, benefit_matrix):
    for i in range(N):
        for j in range(i+1, N):
            if not graph.has_edge(i, j):
                if i == L or i == M or j == M or j == L:
                    benefit_matrix.append((i, j, compute_benefit(graph, i, j)))
    benefit_matrix.sort(key=lambda x: x[2], reverse=True)# sort te updated matrix


In [37]:
#below is the main function to run the algorithm and it should return the final network after building the recommended roads
def main():
    #generate a random graph with aprox .05 connectivity
    graph = nx.random_graphs.erdos_renyi_graph(N, p)
    all_roads = []#all possible roads empty list

    for segment in [(0, 2), (0, 3), (1, 2), (1, 4), (2, 3)]:#provided network
        benefit = compute_benefit(graph, segment[0], segment[1])
        all_roads.append((segment, round(benefit, 2)))#rounding benifit value

    all_roads.sort(key=lambda x: x[1], reverse=True)
    recommended_roads = all_roads[:k]# gettng the first 3 recommended segments based on budget
    
    return graph, recommended_roads, all_roads

# Run the main function
final_network, recommended_roads, all_roads = main()

print("Recommended Roads for construction:")
for segment, benefit in recommended_roads:
    print("Segment:", segment, " Benefit:", benefit)

Recommended Roads for construction:
Segment: (0, 2)  Benefit: 139.8
Segment: (1, 2)  Benefit: 124.8
Segment: (2, 3)  Benefit: 97.2


In [38]:
#R2 - road (1,4) and (1,2) would be recommended for construction 
'''
for the below output for R2 solution I placed the main() function as the following:

def main():
    graph=nx.nxconnected_watts_strogatz_graph(N,4,P) 

    for u,v in [(0,1),(0,4),(1,3),(2,4),(3,4)]:
        graph.add_edge(u,v,weight=random.randint(L_min,L_max))
    all_roads=[]

    for segment in [(0,2),(0,3),(1,2),(1,4),(2,3)]:
        benifit=compute_benifit(graph,segment[0],segment[1])
        all_roads.append((segment,round(benifit,2)))

    all_roads.sort(key=lambda x:x[1],reverse=True)
    recommended_roads = all_roads[:2]

    return graph,recommended_roads,all_roads

final_network,recommended_roads,all_roads = main()
print("final network edges",final_network.edges())
print("recommended roads:")
for segment,benifit in recommended_roads:
    print("Segment:",segment,"benig=fit:",benifit)

'''
'''Final Network Edges: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4)]
Recommended Segments for Construction:
Segment: (1, 4) | Benefit: 103.2
Segment: (1, 2) | Benefit: 55.0

Overall Benefit Factors for All Segments Considered:
Segment: (1, 4) | Benefit: 103.2
Segment: (1, 2) | Benefit: 55.0
Segment: (0, 3) | Benefit: 53.4
Segment: (0, 2) | Benefit: 46.0
Segment: (2, 3) | Benefit: 44.6'''

'Final Network Edges: [(0, 1), (0, 2), (0, 3), (0, 4), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4)]\nRecommended Segments for Construction:\nSegment: (1, 4) | Benefit: 103.2\nSegment: (1, 2) | Benefit: 55.0\n\nOverall Benefit Factors for All Segments Considered:\nSegment: (1, 4) | Benefit: 103.2\nSegment: (1, 2) | Benefit: 55.0\nSegment: (0, 3) | Benefit: 53.4\nSegment: (0, 2) | Benefit: 46.0\nSegment: (2, 3) | Benefit: 44.6'

In [40]:
#Answers to R3,R4,and R5
'''
R4 : (a,b)
roads=segment these are the roads to be constructed(10th time ran through)
Segment: (2, 3)  Benefit: 577.8
Segment: (0, 2)  Benefit: 507.2
Segment: (1, 2)  Benefit: 438.2

c) the recommendations of the roads would change given that the factorization number changed to .8
because originally I had it at .6 then the shrinkage factor for computing the direct distance between nodes is increased. 
This will result in shorter direc distances between nodes and will affect the benifit factor.

R5: If I double the connectivity used in R3 (p=.1) then if the simulation runs again we get different results
below is the following results(first time ran through):

Recommended Roads for construction:
Segment: (0, 3)  Benefit: 156.0
Segment: (0, 2)  Benefit: 118.2
Segment: (1, 2)  Benefit: 103.6
Higher connectivity just means that each node will have more edges.
Each node in the network are more likely to be connected to other nodes. Traffic will also flow smoother with more connections.
So, it makes sense that the benifit value shruck given the increase of options.
'''


'\nR4 : (a,b)\nroads=segment these are the roads to be constructed(10th time ran through)\nSegment: (2, 3)  Benefit: 577.8\nSegment: (0, 2)  Benefit: 507.2\nSegment: (1, 2)  Benefit: 438.2\n\nc) the recommendations of the roads would change given that the factorization number changed to .8\nbecause originally I had it at .6 then the shrinkage factor for computing the direct distance between nodes is increased. \nThis will result in shorter direc distances between nodes and will affect the benifit factor.\n\nR5: If I double the connectivity used in R3 (p=.1) then if the simulation runs again we get different results\nbelow is the following results(first time ran through):\n\nRecommended Roads for construction:\nSegment: (0, 3)  Benefit: 156.0\nSegment: (0, 2)  Benefit: 118.2\nSegment: (1, 2)  Benefit: 103.6\nHigher connectivity just means that each node will have more edges.\nEach node in the network are more likely to be connected to other nodes. Traffic will also flow smoother with mo